In [157]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

In [158]:
from scipy import stats
from scipy.stats import norm,skew
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict


In [159]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

In [160]:
train_data = pd.read_excel("Data_Train.xlsx")
test_data = pd.read_excel("Test_set.xlsx")

In [161]:
big_df = train_data.append(test_data,sort=False)

In [162]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time  Arrival_Time Duration Total_Stops Additional_Info    Price  
0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897.0  
1    05:50         13:15   7h 25m     2 stops         No info   7662.0  
2    09:25  04:25 10 Jun      19h     2 stops         No info  13882.0  
3    18:05         23:30   5h 25m      1 stop         No info   6218.0  
4    16:50         21:35   4h 45m      1 stop         No info  13302.0

In [163]:
big_df["Journey_Date"] = big_df["Date_of_Journey"].str.split('/').str[0]
big_df["Journey_Month"] = big_df["Date_of_Journey"].str.split('/').str[1]
big_df["Journey_Year"] = big_df["Date_of_Journey"].str.split('/').str[2]

In [164]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time  Arrival_Time Duration Total_Stops Additional_Info    Price  \
0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897.0   
1    05:50         13:15   7h 25m     2 stops         No info   7662.0   
2    09:25  04:25 10 Jun      19h     2 stops         No info  13882.0   
3    18:05         23:30   5h 25m      1 stop         No info   6218.0   
4    16:50         21:35   4h 45m      1 stop         No info  13302.0   

  Journey_Date Journey_Month Journey_Year  
0           24            03         2019  
1            1            05         2019  
2            9            06         2019  
3           12            05         2019  
4           01            03         2019

In [165]:
big_df['Arrival_Time'] = big_df['Arrival_Time'].str.split(' ').str[0]

In [166]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time Arrival_Time Duration Total_Stops Additional_Info    Price  \
0    22:20        01:10   2h 50m    non-stop         No info   3897.0   
1    05:50        13:15   7h 25m     2 stops         No info   7662.0   
2    09:25        04:25      19h     2 stops         No info  13882.0   
3    18:05        23:30   5h 25m      1 stop         No info   6218.0   
4    16:50        21:35   4h 45m      1 stop         No info  13302.0   

  Journey_Date Journey_Month Journey_Year  
0           24            03         2019  
1            1            05         2019  
2            9            06         2019  
3           12            05         2019  
4           01            03         2019

In [167]:
big_df['Arrival_Hour'] = big_df['Arrival_Time'].str.split(':').str[0]

In [168]:
big_df['Arrival_Minute'] = big_df['Arrival_Time'].str.split(':').str[1]

In [169]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time Arrival_Time Duration Total_Stops Additional_Info    Price  \
0    22:20        01:10   2h 50m    non-stop         No info   3897.0   
1    05:50        13:15   7h 25m     2 stops         No info   7662.0   
2    09:25        04:25      19h     2 stops         No info  13882.0   
3    18:05        23:30   5h 25m      1 stop         No info   6218.0   
4    16:50        21:35   4h 45m      1 stop         No info  13302.0   

  Journey_Date Journey_Month Journey_Year Arrival_Hour Arrival_Minute  
0           24            03         2019           01             10  
1            1            05         2019           13             15  
2            9            06         2019           04             25  
3           12            05         2019           23             30  
4           01            03         2019           21             35

In [170]:
big_df['Arrival_Hour'] = big_df['Arrival_Hour'].astype(int)


In [171]:
big_df['Arrival_Minute'] = big_df['Arrival_Minute'].astype(int)

In [172]:
from collections import Counter

In [173]:
Counter(big_df['Total_Stops'])

Counter({'non-stop': 4340,
         '2 stops': 1899,
         '1 stop': 7056,
         '3 stops': 56,
         nan: 1,
         '4 stops': 2})

In [174]:
big_df['Total_Stops'] = big_df['Total_Stops'].replace('non-stop','0 stop')

In [175]:
big_df['Stops'] = big_df['Total_Stops'].str.split(' ').str[0]

In [176]:
big_df['Stops'] = big_df['Stops'].fillna(0)

In [177]:
big_df['Stops'] = big_df['Stops'].astype(int)

In [178]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time Arrival_Time Duration Total_Stops Additional_Info    Price  \
0    22:20        01:10   2h 50m      0 stop         No info   3897.0   
1    05:50        13:15   7h 25m     2 stops         No info   7662.0   
2    09:25        04:25      19h     2 stops         No info  13882.0   
3    18:05        23:30   5h 25m      1 stop         No info   6218.0   
4    16:50        21:35   4h 45m      1 stop         No info  13302.0   

  Journey_Date Journey_Month Journey_Year  Arrival_Hour  Arrival_Minute  Stops  
0           24            03         2019             1              10      0  
1            1            05         2019            13              15      2  
2            9            06         2019             4              25      2  
3           12            05         2019            23              30      1  
4           01            03         2019            21              35      1

In [179]:
big_df['Dep_Hour']  = big_df['Dep_Time'].str.split(':').str[0]

In [180]:
big_df['Dep_Minute'] = big_df['Dep_Time'].str.split(':').str[1]

In [181]:
big_df['Dep_Hour'] = big_df['Dep_Hour'].astype(int)
big_df['Dep_Minute'] = big_df['Dep_Minute'].astype(int)

In [182]:
Counter(big_df['Route'])

Counter({'BLR → DEL': 1941,
         'CCU → IXR → BBI → BLR': 7,
         'DEL → LKO → BOM → COK': 46,
         'CCU → NAG → BLR': 11,
         'BLR → NAG → DEL': 4,
         'CCU → BLR': 896,
         'BLR → BOM → DEL': 509,
         'DEL → BOM → COK': 3000,
         'DEL → BLR → COK': 287,
         'MAA → CCU': 456,
         'CCU → BOM → BLR': 1232,
         'DEL → AMD → BOM → COK': 246,
         'DEL → PNQ → COK': 49,
         'DEL → CCU → BOM → COK': 59,
         'BLR → COK → DEL': 18,
         'DEL → IDR → BOM → COK': 117,
         'DEL → LKO → COK': 23,
         'CCU → GAU → DEL → BLR': 49,
         'DEL → NAG → BOM → COK': 83,
         'CCU → MAA → BLR': 78,
         'DEL → HYD → COK': 411,
         'CCU → HYD → BLR': 80,
         'DEL → COK': 262,
         'CCU → DEL → BLR': 689,
         'BLR → BOM → AMD → DEL': 10,
         'BOM → DEL → HYD': 43,
         'DEL → MAA → COK': 190,
         'BOM → HYD': 785,
         'DEL → BHO → BOM → COK': 59,
         'DEL → JAI → BOM → COK':

In [183]:
big_df['Route1'] = big_df['Route'].str.split('→ ').str[0]

In [184]:
big_df['Route2'] = big_df['Route'].str.split('→ ').str[1]
big_df['Route3'] = big_df['Route'].str.split('→ ').str[2]
big_df['Route4'] = big_df['Route'].str.split('→ ').str[3]
big_df['Route5'] = big_df['Route'].str.split('→ ').str[4]
big_df['Route6'] = big_df['Route'].str.split('→ ').str[5]

In [186]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time Arrival_Time Duration Total_Stops Additional_Info  ...    \
0    22:20        01:10   2h 50m      0 stop         No info  ...     
1    05:50        13:15   7h 25m     2 stops         No info  ...     
2    09:25        04:25      19h     2 stops         No info  ...     
3    18:05        23:30   5h 25m      1 stop         No info  ...     
4    16:50        21:35   4h 45m      1 stop         No info  ...     

   Arrival_Minute Stops Dep_Hour Dep_Minute  Route1  Route2  Route3  Route4  \
0              10     0       22         20    BLR      DEL     NaN     NaN   
1              15     2        5         50    CCU     IXR     BBI      BLR   
2              25     2        9         25    DEL     LKO     BOM      COK   
3              30     1       18          5    CCU     NAG      BLR     NaN   
4              35     1       16         50    BLR     NAG      DEL     NaN   

   Route5 Route6  
0     NaN    NaN  
1     NaN    NaN  
2     NaN    NaN  
3     NaN    NaN  
4     NaN    NaN  

[5 rows x 25 columns]

In [187]:
big_df['Route1'].fillna("None", inplace = True)
big_df['Route2'].fillna("None", inplace = True)
big_df['Route3'].fillna("None", inplace = True)
big_df['Route4'].fillna("None", inplace = True)
big_df['Route5'].fillna("None", inplace = True)
big_df['Route6'].fillna("None", inplace = True)

In [188]:
big_df.head()

Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time Arrival_Time Duration Total_Stops Additional_Info  ...    \
0    22:20        01:10   2h 50m      0 stop         No info  ...     
1    05:50        13:15   7h 25m     2 stops         No info  ...     
2    09:25        04:25      19h     2 stops         No info  ...     
3    18:05        23:30   5h 25m      1 stop         No info  ...     
4    16:50        21:35   4h 45m      1 stop         No info  ...     

   Arrival_Minute Stops Dep_Hour Dep_Minute  Route1  Route2  Route3  Route4  \
0              10     0       22         20    BLR      DEL    None    None   
1              15     2        5         50    CCU     IXR     BBI      BLR   
2              25     2        9         25    DEL     LKO     BOM      COK   
3              30     1       18          5    CCU     NAG      BLR    None   
4              35     1       16         50    BLR     NAG      DEL    None   

   Route5 Route6  
0    None   None  
1    None   None  
2    None   None  
3    None   None  
4    None   None  

[5 rows x 25 columns]

In [189]:
cols = ['Date_of_Journey','Route','Dep_Time', 'Arrival_Time','Duration','Total_Stops']

big_df.drop(cols,axis=1,inplace= True)

In [190]:
big_df

Airline    Source Destination               Additional_Info  \
0                IndiGo  Banglore   New Delhi                       No info   
1             Air India   Kolkata    Banglore                       No info   
2           Jet Airways     Delhi      Cochin                       No info   
3                IndiGo   Kolkata    Banglore                       No info   
4                IndiGo  Banglore   New Delhi                       No info   
5              SpiceJet   Kolkata    Banglore                       No info   
6           Jet Airways  Banglore   New Delhi   In-flight meal not included   
7           Jet Airways  Banglore   New Delhi                       No info   
8           Jet Airways  Banglore   New Delhi   In-flight meal not included   
9     Multiple carriers     Delhi      Cochin                       No info   
10            Air India     Delhi      Cochin                       No info   
11               IndiGo   Kolkata    Banglore                       No info   
12            Air India   Chennai     Kolkata                       No info   
13          Jet Airways   Kolkata    Banglore   In-flight meal not included   
14               IndiGo   Kolkata    Banglore                       No info   
15            Air India     Delhi      Cochin                       No info   
16             SpiceJet     Delhi      Cochin                       No info   
17          Jet Airways     Delhi      Cochin   In-flight meal not included   
18            Air India     Delhi      Cochin                       No info   
19          Jet Airways     Delhi      Cochin   In-flight meal not included   
20                GoAir     Delhi      Cochin                       No info   
21            Air India  Banglore   New Delhi                       No info   
22               IndiGo  Banglore       Delhi                       No info   
23               IndiGo  Banglore       Delhi                       No info   
24          Jet Airways   Kolkata    Banglore   In-flight meal not included   
25          Jet Airways     Delhi      Cochin                       No info   
26               IndiGo     Delhi      Cochin                       No info   
27                GoAir     Delhi      Cochin                       No info   
28              Vistara  Banglore       Delhi                       No info   
29              Vistara   Chennai     Kolkata                       No info   
...                 ...       ...         ...                           ...   
2641  Multiple carriers     Delhi      Cochin                       No info   
2642        Jet Airways     Delhi      Cochin   In-flight meal not included   
2643          Air India     Delhi      Cochin                       No info   
2644  Multiple carriers     Delhi      Cochin                       No info   
2645        Jet Airways  Banglore   New Delhi   In-flight meal not included   
2646        Jet Airways     Delhi      Cochin                       No info   
2647          Air India   Kolkata    Banglore                       No info   
2648             IndiGo     Delhi      Cochin                       No info   
2649           SpiceJet  Banglore       Delhi  No check-in baggage included   
2650          Air India   Kolkata    Banglore                       No info   
2651        Jet Airways   Kolkata    Banglore   In-flight meal not included   
2652        Jet Airways  Banglore   New Delhi                       No info   
2653        Jet Airways   Kolkata    Banglore   In-flight meal not included   
2654          Air India     Delhi      Cochin                       No info   
2655  Multiple carriers     Delhi      Cochin   In-flight meal not included   
2656        Jet Airways   Kolkata    Banglore   In-flight meal not included   
2657  Multiple carriers     Delhi      Cochin                       No info   
2658        Jet Airways     Delhi      Cochin   In-flight meal not included   
2659        Jet Airways     Delhi      Cochin   In-flight meal not

In [191]:
from sklearn.preprocessing import LabelEncoder

In [192]:
le = LabelEncoder()
big_df['Airline'] = le.fit_transform(big_df['Airline'])
big_df['Source'] = le.fit_transform(big_df['Source'])
big_df['Destination'] = le.fit_transform(big_df['Destination'])
big_df['Additional_Info'] = le.fit_transform(big_df['Additional_Info'])
big_df['Route1'] = le.fit_transform(big_df['Route1'])
big_df['Route2'] = le.fit_transform(big_df['Route2'])
big_df['Route3'] = le.fit_transform(big_df['Route3'])
big_df['Route4'] = le.fit_transform(big_df['Route4'])
big_df['Route5'] = le.fit_transform(big_df['Route5'])
big_df['Route6'] = le.fit_transform(big_df['Route6'])

In [193]:
df_train = big_df[0:10683]
df_test = big_df[10683:]
df_test = df_test.drop(['Price'], axis =1)

In [194]:
X_train = df_train.drop(['Price'], axis = 1)
y_train = df_train['Price']

In [200]:
lm = LinearRegression()

kfolds = KFold(n_splits=50,shuffle=True,random_state=100)

In [202]:
def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model,X_train,y_train, scoring = "neg_mean_squared_error",cv=kfolds))
    return(rmse)

In [203]:
benchmarkmodel = make_pipeline(RobustScaler(),lm).fit(X = X_train,y=y_train)

In [204]:
cv_rmse(benchmarkmodel).mean()

3239.0465651403265

In [209]:
y_predict = model.predict(df_test.head(2))

In [210]:
y_predict

array([11009.0873725 ,  8151.84507445])